In [7]:
!python --version
import warnings

warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

Python 3.11.7


In [3]:
import torch
torch.__version__

'2.1.2+cpu'

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler , OneHotEncoder

In [10]:
insurance_data = pd.read_csv("datasets/insurance.csv", on_bad_lines='skip')

insurance_data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [11]:
X = insurance_data.drop(columns = ["charges"])
y = insurance_data["charges"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 123)

X_train.shape, X_val.shape, y_train.shape, y_val.shape
categorical_features = ["sex", "smoker", "region"]

categorical_transformer = OneHotEncoder(
    handle_unknown = "ignore", drop = "first", sparse_output = False
)

preprocessor = ColumnTransformer(
    transformers = [("cat_tr", categorical_transformer, categorical_features)],
    remainder = "passthrough"
)
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)


pd.DataFrame(X_train, columns = preprocessor.get_feature_names_out())
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()
stdscaler = StandardScaler()

X_train = stdscaler.fit_transform(X_train)
X_val = stdscaler.transform(X_val)

min_max_scaler = MinMaxScaler()

y_train = min_max_scaler.fit_transform(y_train.reshape(-1, 1))

y_val = min_max_scaler.transform(y_val.reshape(-1, 1))
train_inputs = torch.from_numpy(X_train).float()
train_targets = torch.from_numpy(y_train.reshape(-1, 1)).float()



((1070, 6), (268, 6), (1070,), (268,))

In [20]:
class SimpleNeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 1)

    def forward(self, x_in):
        y_pred = self.fc1(x_in)
        return y_pred

In [21]:
model = SimpleNeuralNetwork(input_dim = 8)

In [22]:
import torch.nn.functional as F

loss_fn = F.mse_loss

In [24]:
loss = loss_fn(model(train_inputs), train_targets)

print(loss)

tensor(0.4929, grad_fn=<MseLossBackward0>)


In [25]:
preds = model(train_inputs)

preds

tensor([[ 0.0946],
        [-0.8909],
        [-0.4565],
        ...,
        [ 0.0211],
        [ 0.3600],
        [-0.4017]], grad_fn=<AddmmBackward0>)

In [26]:
from torchmetrics.regression import R2Score
from torchmetrics.regression import MeanSquaredError

MSE = MeanSquaredError()

r2score = R2Score()

print("Mean Squared Error :", MSE(preds, train_targets).item())
print("R^2 :", r2score(preds, train_targets).item())

Mean Squared Error : 0.4929397702217102
R^2 : -11.853158950805664


In [28]:
from torch.utils.data import TensorDataset, DataLoader
train_df = TensorDataset(train_inputs, train_targets)

In [30]:
train_df[:5]

(tensor([[ 0.9888, -0.5000,  1.7213, -0.6213, -0.5478,  0.0621, -0.7196, -0.0674],
         [-1.0113,  2.0000, -0.5809,  1.6095, -0.5478, -0.1506,  1.2870, -0.8865],
         [ 0.9888, -0.5000, -0.5809, -0.6213,  1.8254,  0.7709, -0.6722, -0.8865],
         [ 0.9888, -0.5000, -0.5809, -0.6213, -0.5478,  0.4874, -0.9722,  1.5709],
         [-1.0113, -0.5000, -0.5809, -0.6213,  1.8254, -1.2847, -2.2011, -0.0674]]),
 tensor([[0.0857],
         [0.6393],
         [0.1191],
         [0.1363],
         [0.0238]]))

In [31]:
batch_size = 8

In [32]:
train_df = DataLoader(train_df, batch_size = batch_size, shuffle = True)

In [33]:
next(iter(train_df))

[tensor([[-1.0113, -0.5000,  1.7213, -0.6213, -0.5478, -0.6468, -1.7773,  1.5709],
         [-1.0113, -0.5000,  1.7213, -0.6213, -0.5478, -0.8594, -1.5247, -0.8865],
         [ 0.9888, -0.5000, -0.5809, -0.6213, -0.5478, -0.2923,  1.5063,  2.3900],
         [ 0.9888, -0.5000, -0.5809,  1.6095, -0.5478, -0.0088,  0.5741,  0.7517],
         [-1.0113,  2.0000, -0.5809, -0.6213, -0.5478,  0.5582, -0.7512, -0.0674],
         [-1.0113, -0.5000, -0.5809,  1.6095, -0.5478, -0.9303, -0.1937, -0.0674],
         [-1.0113, -0.5000, -0.5809, -0.6213, -0.5478, -1.4973, -0.0881, -0.8865],
         [ 0.9888, -0.5000, -0.5809,  1.6095, -0.5478, -1.1429,  3.2794, -0.0674]]),
 tensor([[0.0744],
         [0.0363],
         [0.2989],
         [0.3651],
         [0.3624],
         [0.0369],
         [0.0176],
         [0.0214]])]

In [34]:
val_inputs = torch.from_numpy(X_val).float()
val_targets = torch.from_numpy(y_val.reshape(-1, 1)).float()

In [35]:
val_df = TensorDataset(val_inputs, val_targets)

In [36]:
val_df[:5]

(tensor([[-1.0113, -0.5000, -0.5809,  1.6095, -0.5478,  0.7000,  1.9999,  0.7517],
         [ 0.9888, -0.5000, -0.5809, -0.6213, -0.5478, -0.5050,  1.1117, -0.0674],
         [-1.0113,  2.0000, -0.5809, -0.6213,  1.8254, -0.8594,  0.1254, -0.8865],
         [ 0.9888, -0.5000,  1.7213, -0.6213, -0.5478, -0.2923, -1.0827, -0.0674],
         [ 0.9888, -0.5000, -0.5809,  1.6095, -0.5478,  1.4797, -0.8152, -0.8865]]),
 tensor([[0.1412],
         [0.0577],
         [0.5485],
         [0.0651],
         [0.1793]]))

In [37]:
val_df = DataLoader(val_df, batch_size = batch_size, shuffle = True)

In [39]:
next(iter(val_df))

[tensor([[ 0.9888,  2.0000, -0.5809, -0.6213, -0.5478,  0.1330,  0.0224,  1.5709],
         [ 0.9888, -0.5000,  1.7213, -0.6213, -0.5478,  1.4088, -0.8617, -0.8865],
         [-1.0113, -0.5000, -0.5809, -0.6213,  1.8254,  1.6215,  0.4245, -0.8865],
         [-1.0113, -0.5000, -0.5809, -0.6213, -0.5478, -1.2847, -1.4142, -0.8865],
         [ 0.9888, -0.5000,  1.7213, -0.6213, -0.5478, -0.3632, -0.8933, -0.0674],
         [-1.0113, -0.5000, -0.5809, -0.6213,  1.8254,  0.0621, -0.1737, -0.8865],
         [-1.0113, -0.5000, -0.5809, -0.6213, -0.5478,  0.9835, -1.1459,  0.7517],
         [-1.0113,  2.0000, -0.5809, -0.6213,  1.8254, -0.8594,  0.1254, -0.8865]]),
 tensor([[0.6259],
         [0.1790],
         [0.2008],
         [0.0238],
         [0.0614],
         [0.0779],
         [0.1726],
         [0.5485]])]

In [41]:
loss_stats = {
    "train": [],
    "val": []
}

num_epochs = 100

In [40]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cpu device


In [45]:
num_of_epochs = 100
optimizer = optim.Adam(model.parameters(), lr = 0.01)

for i in range(num_of_epochs):
    train_epoch_loss = 0
    model.train()
    
    for X_train_batch, y_train_batch in train_df:
        optimizer.zero_grad()
        
        X_train_batch, y_train_batch = \
            X_train_batch.to(device), y_train_batch.to(device)
        
        preds = model(X_train_batch)
        
        train_loss= loss_fn(preds, y_train_batch)
        
        train_loss.backward()
        
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        
    with torch.no_grad():
        val_epoch_loss = 0
        model.eval()
        
        for X_val_batch, y_val_batch in val_df:
            preds = model(X_val_batch)
            
            val_loss = loss_fn(preds, y_val_batch)
            
            val_epoch_loss += val_loss.item()
            
    train_loss = train_epoch_loss / len(train_df)
    val_loss = val_epoch_loss / len(val_df)
        
    loss_stats["train"].append(train_loss)
    loss_stats["val"].append(val_loss)

    print(f'Epoch {i+0:01}: | Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f}')
        

Epoch 0: | Train Loss: 0.012 | Val Loss: 0.011
Epoch 1: | Train Loss: 0.012 | Val Loss: 0.008
Epoch 2: | Train Loss: 0.012 | Val Loss: 0.011
Epoch 3: | Train Loss: 0.012 | Val Loss: 0.009
Epoch 4: | Train Loss: 0.012 | Val Loss: 0.008
Epoch 5: | Train Loss: 0.012 | Val Loss: 0.010
Epoch 6: | Train Loss: 0.012 | Val Loss: 0.011
Epoch 7: | Train Loss: 0.012 | Val Loss: 0.008
Epoch 8: | Train Loss: 0.012 | Val Loss: 0.011
Epoch 9: | Train Loss: 0.012 | Val Loss: 0.008
Epoch 10: | Train Loss: 0.012 | Val Loss: 0.010
Epoch 11: | Train Loss: 0.012 | Val Loss: 0.010
Epoch 12: | Train Loss: 0.012 | Val Loss: 0.011
Epoch 13: | Train Loss: 0.011 | Val Loss: 0.010
Epoch 14: | Train Loss: 0.012 | Val Loss: 0.009
Epoch 15: | Train Loss: 0.012 | Val Loss: 0.009
Epoch 16: | Train Loss: 0.012 | Val Loss: 0.008
Epoch 17: | Train Loss: 0.012 | Val Loss: 0.009
Epoch 18: | Train Loss: 0.012 | Val Loss: 0.010
Epoch 19: | Train Loss: 0.011 | Val Loss: 0.009
Epoch 20: | Train Loss: 0.012 | Val Loss: 0.009
Ep